In [2]:
import matplotlib.pyplot as plt
import numpy as np

from l5kit.data import ChunkedDataset, LocalDataManager
from l5kit.dataset import EgoDataset, AgentDataset
from l5kit.rasterization import build_rasterizer
from l5kit.configs import load_config_data

import os

In [3]:
# opening the zarr_dataset
os.environ["L5KIT_DATA_FOLDER"] = "../../prediction-dataset/"
cfg = load_config_data("./visualisation_config.yaml")
dm = LocalDataManager()
dataset_path = dm.require(cfg["val_data_loader"]["key"])
zarr_dataset = ChunkedDataset(dataset_path)
zarr_dataset.open()

In [4]:
# using EgoDataset interface to extract AV data
rast = build_rasterizer(cfg, dm)
ego_dataset = EgoDataset(cfg, zarr_dataset, rast)
# agent_dataset = AgentDataset(cfg, zarr_dataset, rast)

In [16]:
# checking which frames belong to the scene 0 to know which frames to analyze
print(zarr_dataset.scenes[0])
# for i in range(len(zarr_dataset.scenes)):
#     print(zarr_dataset.scenes[i][0][0], zarr_dataset.scenes[i][0][1])

([  0, 248], 'host-a013', 1572643684617362176, 1572643709617362176)


In [19]:
# scene_frame_indexes = []
# for i in range(len(zarr_dataset.scenes)):
#     scene_frame_indexes.append([zarr_dataset.scenes[i][0][0], zarr_dataset.scenes[i][0][1]])

# print(scene_frame_indexes)

[[0, 248], [248, 497], [497, 746], [746, 995], [995, 1244], [1244, 1493], [1493, 1742], [1742, 1991], [1991, 2240], [2240, 2489], [2489, 2738], [2738, 2987], [2987, 3236], [3236, 3485], [3485, 3734], [3734, 3983], [3983, 4232], [4232, 4481], [4481, 4730], [4730, 4979], [4979, 5228], [5228, 5477], [5477, 5725], [5725, 5973], [5973, 6221], [6221, 6469], [6469, 6717], [6717, 6965], [6965, 7213], [7213, 7461], [7461, 7709], [7709, 7957], [7957, 8205], [8205, 8453], [8453, 8701], [8701, 8949], [8949, 9197], [9197, 9445], [9445, 9693], [9693, 9941], [9941, 10189], [10189, 10437], [10437, 10685], [10685, 10933], [10933, 11182], [11182, 11431], [11431, 11680], [11680, 11929], [11929, 12177], [12177, 12425], [12425, 12673], [12673, 12921], [12921, 13169], [13169, 13417], [13417, 13665], [13665, 13913], [13913, 14161], [14161, 14409], [14409, 14657], [14657, 14905], [14905, 15153], [15153, 15401], [15401, 15649], [15649, 15897], [15897, 16145], [16145, 16393], [16393, 16641], [16641, 16889], [16

In [6]:
# extracting position information from ego dataset
position = []
# low = scene_frame_indexes[0][0]
# high = scene_frame_indexes[0][1]
for i in range(0, 248):
    position.append(ego_dataset[i]["centroid"])

In [7]:
#calculating variance and standard deviation of x and y position data

xsum = 0
ysum = 0

for i in range(len(position)):
    xsum += position[i][0]
    ysum += position[i][1]

xsum /= len(position)
ysum /= len(position)

xvar = 0
yvar = 0

for i in range(len(position)):
    xvar += ((position[i][0] - xsum)**2)
    yvar += ((position[i][1] - ysum)**2)

xvar /= len(position) - 1
yvar /= len(position) - 1

print("x variance:", xvar, ", y variance:", yvar)
print("x std dev:", xvar**0.5, "y std dev:", yvar**0.5)

x variance: 2759.868110530938 , y variance: 6374.833443842314
x std dev: 52.534446894689374 y std dev: 79.84255409142618


In [8]:
# setting up matrices for Kalman Filter
# initial state matrix, "unknown" vehicle location so we set pos, vel, acc to 0
x = np.array([[0],
              [0],
              [0],
              [0],
              [0],
              [0]])

# since velocity and acceleration is unkown, a high estimate uncertainty is set, resulting in a high Kalman Gain
P = np.array([[500, 0, 0, 0, 0, 0],
              [0, 500, 0, 0, 0, 0],
              [0, 0, 500, 0, 0, 0],
              [0, 0, 0, 500, 0, 0],
              [0, 0, 0, 0, 500, 0],
              [0, 0, 0, 0, 0, 500]])

# t = measurement period / delta time in seconds
t = 1
acc_std_dev = 0.2

# state transition matrix, kinematic equations for position, velocity, and acceleration
F = np.array([[1, t, 0.5*(t**2), 0, 0,          0], 
              [0, 1,          t, 0, 0,          0], 
              [0, 0,          1, 0, 0,          0], 
              [0, 0,          0, 1, t, 0.5*(t**2)], 
              [0, 0,          0, 0, 1,          t], 
              [0, 0,          0, 0, 0,          1]])

# process noise matrix
Q = np.array([[0.25, 0.5, 0.5,    0,   0,   0], 
              [ 0.5,   1,   1,    0,   0,   0], 
              [ 0.5,   1,   1,    0,   0,   0],
              [   0,   0,   0, 0.25, 0.5, 0.5],
              [   0,   0,   0,  0.5,   1,   1],
              [   0,   0,   0,  0.5,   1,   1]]) * (acc_std_dev**2)

# measurement uncertainty matrix
R = np.array([[xvar, 0], 
              [0, yvar]])

H = np.array([[1, 0, 0, 0, 0, 0], 
              [0, 0, 0, 1, 0, 0]])

I = np.array([[1, 0, 0, 0, 0, 0],
              [0, 1, 0, 0, 0, 0],
              [0, 0, 1, 0, 0, 0],
              [0, 0, 0, 1, 0, 0],
              [0, 0, 0, 0, 1, 0],
              [0, 0, 0, 0, 0, 1]])

In [9]:
# initialization
x = np.matmul(F, x)
P = np.matmul(np.matmul(F, P), np.transpose(F)) + Q
print(x)
print(P)

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[1125.01  750.02  250.02    0.      0.      0.  ]
 [ 750.02 1000.04  500.04    0.      0.      0.  ]
 [ 250.02  500.04  500.04    0.      0.      0.  ]
 [   0.      0.      0.   1125.01  750.02  250.02]
 [   0.      0.      0.    750.02 1000.04  500.04]
 [   0.      0.      0.    250.02  500.04  500.04]]


In [23]:
def kalmanFilter(measurements, iterations, x, P, R, H, Q, I):
    for i in range(iterations):
        z = np.array([[measurements[i][0]],
                      [measurements[i][1]]])
        
        # Kalman Gain calculation
        # K = (P * H_T) * (H * P * H_T + R)^-1
        K = np.matmul(np.matmul(P, np.transpose(H)), 
                      np.linalg.inv(np.matmul(np.matmul(H, P), np.transpose(H)) + R))

        # current state estimation
        # x = x + K * (z - H * x)
        x = x + np.matmul(K, (z - np.matmul(H, x)))

        # update estimate uncertainty
        # P = (I - K * H) * P * (I - K * H)_T + K * R * K_T
        # IKH = (I - K * H)
        IKH = I - np.matmul(K, H)
        P = np.matmul(np.matmul(IKH, P), np.transpose(IKH)) + np.matmul(np.matmul(K, R), np.transpose(K))

        # predict state and update estimate uncertainty
        # x = F * x
        # P = F * P * F_T + Q
        x = np.matmul(F, x)
        P = np.matmul(np.matmul(F, P), np.transpose(F)) + Q 
    return (x, P, R, H, Q)

In [35]:
kalmanFilter(measurements=position, iterations=10, x=x, P=P, R=R, H=H, Q=Q, I=I)

(array([[  570.17622605],
        [  -88.06447995],
        [  -23.18684203],
        [-2169.51262827],
        [  121.69639899],
        [   53.26295786]]),
 array([[2715.51710142,  826.31605357,  107.07101473,    0.        ,
            0.        ,    0.        ],
        [ 826.31605357,  311.05098261,   44.23601048,    0.        ,
            0.        ,    0.        ],
        [ 107.07101473,   44.23601048,    6.66678403,    0.        ,
            0.        ,    0.        ],
        [   0.        ,    0.        ,    0.        , 5682.01885384,
         1620.3607484 ,  201.03095817],
        [   0.        ,    0.        ,    0.        , 1620.3607484 ,
          575.32097591,   78.5477166 ],
        [   0.        ,    0.        ,    0.        ,  201.03095817,
           78.5477166 ,   11.2731174 ]]),
 array([[2759.86811053,    0.        ],
        [   0.        , 6374.83344384]]),
 array([[1, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0]]),
 array([[0.01, 0.02, 0.02, 0.  , 0.  , 0.  ],


In [36]:
position[10]

array([  686.55358887, -2174.33203125])